In [1]:
import pybullet as p
import pybullet_data
import time

def main():
    # 1. Koneksi ke PyBullet (GUI agar bisa melihat visualisasi)
    p.connect(p.GUI)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())  # Untuk memudahkan akses file default

    # 2. Set gravitasi dan non-aktifkan real-time agar kita bisa kontrol step
    p.setGravity(0, 0, -9.81)
    p.setRealTimeSimulation(0)

    # 3. Load lantai (plane) dan URDF roket
    p.loadURDF("plane.urdf")
    
    # Pastikan path URDF rocket Anda benar. Misal rocket.urdf ada di folder yang sama:
    rocket_urdf_path = "LunarLander3DEnv/envs/urdf/rocket.urdf"
    rocket_start_pos = [0, 0, 0]
    rocket_start_orn = p.getQuaternionFromEuler([0, 0, 0])
    rocket_id = p.loadURDF(
        rocket_urdf_path,
        rocket_start_pos,
        rocket_start_orn,
        useFixedBase=False  # Jika roket mau bergerak bebas
    )

    # 4. Identifikasi indeks joint (plate_joint dan engine_joint)
    plate_joint_idx = None
    engine_joint_idx = None

    num_joints = p.getNumJoints(rocket_id)
    for i in range(num_joints):
        info = p.getJointInfo(rocket_id, i)
        joint_name = info[1].decode("utf-8")
        if joint_name == "plate_joint":
            plate_joint_idx = i
        elif joint_name == "engine_joint":
            engine_joint_idx = i
    
    # 5. (Opsional) Cari link index untuk main engine dan RCS jika ingin menempatkan thruster per link
    #    Anda bisa melihat nama link di URDF: "engine" dan "rcs".
    #    Di PyBullet, link index = joint index (untuk child link). Link 0 biasanya base link (jika multi).
    main_engine_link_idx = None
    rcs_link_idx = None
    for i in range(num_joints):
        info = p.getJointInfo(rocket_id, i)
        link_name = info[12].decode("utf-8")  # child link name
        if link_name == "engine":
            main_engine_link_idx = i
        elif link_name == "rcs":
            rcs_link_idx = i
    
    print(f" link engine :{main_engine_link_idx}")
    print(f" link rcs :{rcs_link_idx}")
    # 6. Fungsi untuk menerapkan thrust (gaya) di link tertentu
    #    force_dir: vektor [Fx, Fy, Fz] di kerangka link (LINK_FRAME) atau world (WORLD_FRAME)
    #    pos_offset: posisi di mana gaya diaplikasikan (dalam kerangka link jika LINK_FRAME)
    def apply_thrust(body_id, link_id, force_dir, pos_offset=[0,0,0], frame=p.LINK_FRAME):
        p.applyExternalForce(
            objectUniqueId=body_id,
            linkIndex=link_id,
            forceObj=force_dir,
            posObj=pos_offset,
            flags=frame
        )
    
    # 7. Contoh penempatan empat thruster RCS di sekeliling link rcs (kiri, kanan, depan, belakang).
    #    Asumsi rcs adalah plat bundar, dengan radius ~0.05 m. 
    #    Silakan atur offset sesuai desain Anda.
    rcs_offsets = [
        [0.05, 0.0, 0.0],   # RCS kanan
        [-0.05, 0.0, 0.0],  # RCS kiri
        [0.0, 0.05, 0.0],   # RCS depan
        [0.0, -0.05, 0.0],  # RCS belakang
    ]
    
    # 8. Loop simulasi
    #    Kita contohkan:
    #    - Set sudut plate_joint (sumbu X) dan engine_joint (sumbu Y) secara manual
    #    - Berikan thrust konstan di main engine (arah +Z di link engine)
    #    - Berikan thrust kecil di 4 RCS (arah +/-Z untuk manuver lateral)
    plate_angle = -.5
    engine_angle = .0
    main_engine_thrust = 00  # Newton
    rcs_thrust = 0.0           # Newton (kecil untuk kontrol)

    # Gunakan mode kontrol posisi untuk joint
    p.setJointMotorControl2(
        rocket_id, plate_joint_idx,
        controlMode=p.POSITION_CONTROL,
        targetPosition=plate_angle,
        force=10
    )
    p.setJointMotorControl2(
        rocket_id, engine_joint_idx,
        controlMode=p.POSITION_CONTROL,
        targetPosition=engine_angle,
        force=10
    )

    while True:
        # 8a. Terapkan thrust pada main engine
        if main_engine_link_idx is not None:
            # Arah thrust di main engine: misal, ke arah +Z pada kerangka engine
            apply_thrust(
                rocket_id, 
                main_engine_link_idx, 
                [0, 0, main_engine_thrust], 
                pos_offset=[0, 0, 0], 
                frame=p.LINK_FRAME
            )
        
        # 8b. Terapkan thrust pada 4 RCS sesuai posisi masing-masing
        if rcs_link_idx is not None:
            for offset in rcs_offsets:
                # Tentukan thrust berdasarkan offset:
                # Jika offset di sumbu x, arah thrust adalah -x atau +x.
                # Jika offset di sumbu y, arah thrust adalah -y atau +y.
                if offset[0] != 0:
                    thrust_dir = [-rcs_thrust if offset[0] > 0 else rcs_thrust, 0, 0]
                elif offset[1] != 0:
                    thrust_dir = [0, -rcs_thrust if offset[1] > 0 else rcs_thrust, 0]
                else:
                    thrust_dir = [0, 0, 0]
                
                apply_thrust(
                    rocket_id, 
                    rcs_link_idx, 
                    thrust_dir, 
                    pos_offset=offset, 
                    frame=p.LINK_FRAME
                )

        rocket_pos, _ = p.getBasePositionAndOrientation(rocket_id)
        p.resetDebugVisualizerCamera(cameraDistance=1,cameraYaw=45,cameraPitch=-0,cameraTargetPosition=rocket_pos)
        # 8c. Lakukan step simulasi
        p.stepSimulation()
        time.sleep(1./240.)  # kecepatan simulasi ~240 Hz

    # Jangan lupa p.disconnect() jika diperlukan

if __name__ == "__main__":
    main()


 link engine :1
 link rcs :2


error: Not connected to physics server.

In [1]:
import pybullet as p
import pybullet_data
import time
import math

def main():
    # ----- Setup PyBullet -----
    p.connect(p.GUI)
    p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.81)
    p.setRealTimeSimulation(0)
    
    # Load plane dan URDF roket
    p.loadURDF("plane.urdf")
    rocket_urdf_path = "LunarLander3DEnv/envs/urdf/rocket.urdf"
    # Titik awal: (50,50,50)
    rocket_start_pos = [50, 50, 50]
    # Untuk landing, target adalah (0,0,0)
    # Jika ingin set kemiringan awal 45° pada x dan y, Anda bisa mengubah orientasi base
    # Misalnya: p.getQuaternionFromEuler([0.7854, 0.7854, 0])
    rocket_start_orn = p.getQuaternionFromEuler([0, 0, 0])
    rocket_id = p.loadURDF(
        rocket_urdf_path,
        rocket_start_pos,
        rocket_start_orn,
        useFixedBase=False
    )
    
    # ----- Identifikasi Joint dan Link -----
    plate_joint_idx = None
    engine_joint_idx = None
    num_joints = p.getNumJoints(rocket_id)
    for i in range(num_joints):
        info = p.getJointInfo(rocket_id, i)
        joint_name = info[1].decode("utf-8")
        if joint_name == "plate_joint":
            plate_joint_idx = i
        elif joint_name == "engine_joint":
            engine_joint_idx = i

    main_engine_link_idx = None
    rcs_link_idx = None
    for i in range(num_joints):
        info = p.getJointInfo(rocket_id, i)
        link_name = info[12].decode("utf-8")
        if link_name == "engine":
            main_engine_link_idx = i
        elif link_name == "rcs":
            rcs_link_idx = i

    # ----- Fungsi Apply Thrust -----
    def apply_thrust(body_id, link_id, force_dir, pos_offset=[0, 0, 0], frame=p.LINK_FRAME):
        p.applyExternalForce(
            objectUniqueId=body_id,
            linkIndex=link_id,
            forceObj=force_dir,
            posObj=pos_offset,
            flags=frame
        )
    
    # Offset thruster RCS untuk koreksi horizontal
    # Asumsi rcs berbentuk plat bundar dengan radius ~0.05 m.
    # Untuk mode 2 dan 3, kita akan gunakan posisi offset yang berbeda untuk x dan y.
    rcs_offset_left  = [-0.05, 0, 0]   # menghasilkan gaya +x
    rcs_offset_right = [ 0.05, 0, 0]   # menghasilkan gaya -x
    rcs_offset_front = [0,  0.05, 0]    # menghasilkan gaya -y
    rcs_offset_back  = [0, -0.05, 0]    # menghasilkan gaya +y

    # ----- PID Controller Setup -----
    dt = 1/240.0
    # Inisialisasi error dan integral untuk tiap sumbu
    e_prev = [0.0, 0.0, 0.0]
    integral = [0.0, 0.0, 0.0]
    target = [0.0, 0.0, 0.0]  # target landing di (0,0,0)

    # PID Gains (sesuaikan nilai-nilainya)
    Kp = [0.2, 0.2, 10.0]   # untuk x, y, z
    Ki = [0.0, 0.0, 0.0]
    Kd = [0.05, 0.05, 5.0]

    mass = 10.0  # kg
    # Main engine maksimum (10kg * 2g)
    max_main = mass * 2 * 9.81   # ≈196.2 N
    max_rcs = max_main / 10.0    # ≈19.62 N

    # ----- Mode Kontrol -----
    # Pilih salah satu:
    # control_mode = 1  -> Hanya main engine aktif (dapat diputar), RCS OFF
    # control_mode = 2  -> Main engine fixed (tidak diputar), RCS aktif
    # control_mode = 3  -> Kombinasi: main engine dapat diputar dan RCS aktif
    control_mode = 2

    while True:
        # Dapatkan posisi dan kecepatan roket
        pos, _ = p.getBasePositionAndOrientation(rocket_id)
        vel, _ = p.getBaseVelocity(rocket_id)
        
        # Hitung error posisi
        e = [target[i] - pos[i] for i in range(3)]
        # Hitung derivatif error (approx)
        de = [(e[i] - e_prev[i]) / dt for i in range(3)]
        # Update integral error
        for i in range(3):
            integral[i] += e[i] * dt

        # PID output gaya untuk tiap sumbu
        F_pid = [Kp[i]*e[i] + Kd[i]*de[i] + Ki[i]*integral[i] for i in range(3)]
        # Tambahkan kompensasi gravitasi pada sumbu z
        F_pid[2] += mass * 9.81

        # F_pid = [F_x, F_y, F_z] adalah gaya yang diinginkan untuk menggerakkan roket
        # Berdasarkan mode kontrol, kita alokasikan gaya ke main engine dan RCS.
        if control_mode == 1:
            # ----- Mode 1: Hanya Main Engine Aktif (dapat diputar) -----
            # Semua gaya dikomando melalui main engine.
            F_main_x = F_pid[0]
            F_main_y = F_pid[1]
            F_main_z = F_pid[2]
            F_total = math.sqrt(F_main_x**2 + F_main_y**2 + F_main_z**2)
            # Saturasi jika melebihi kapasitas
            if F_total > max_main:
                scale = max_main / F_total
                F_main_x *= scale
                F_main_y *= scale
                F_main_z *= scale
                F_total = max_main
            # Jika F_total sangat kecil, hindari pembagian nol
            if F_total < 1e-3:
                d = [0, 0, 1]
            else:
                d = [F_main_x/F_total, F_main_y/F_total, F_main_z/F_total]
            # Hitung sudut joint dari arah thrust yang diinginkan.
            # Dengan model: R = R_plate (rotasi sekitar x) * R_engine (rotasi sekitar y)
            # Sehingga:
            #   d_x = sin(engine_angle)
            #   d_y = -cos(engine_angle)*sin(plate_angle)
            #   d_z = cos(engine_angle)*cos(plate_angle)
            # Peroleh:
            engine_angle = math.asin(d[0])
            plate_angle = -math.atan2(d[1], d[2])
            # Set joint main engine
            p.setJointMotorControl2(rocket_id, plate_joint_idx,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=plate_angle, force=10)
            p.setJointMotorControl2(rocket_id, engine_joint_idx,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=engine_angle, force=10)
            # Terapkan thrust pada main engine
            apply_thrust(rocket_id, main_engine_link_idx, [0, 0, F_total], [0, 0, 0], p.LINK_FRAME)
            # RCS tidak aktif pada mode ini.

        elif control_mode == 2:
            # ----- Mode 2: Main Engine Fixed (vertikal) + RCS Aktif -----
            # Tetapkan joint ke nol (tidak diputar)
            p.setJointMotorControl2(rocket_id, plate_joint_idx,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=0, force=10)
            p.setJointMotorControl2(rocket_id, engine_joint_idx,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=0, force=10)
            # Main engine hanya menghasilkan gaya vertikal.
            F_main_vertical = F_pid[2]
            if F_main_vertical > max_main:
                F_main_vertical = max_main
            apply_thrust(rocket_id, main_engine_link_idx, [0, 0, F_main_vertical], [0, 0, 0], p.LINK_FRAME)
            # Koreksi horizontal dengan RCS.
            # Sumbu X:
            if F_pid[0] > 0:
                # Untuk gaya positif di x, gunakan thruster di sisi kiri (offset negatif x)
                thrust = min(F_pid[0], max_rcs)
                apply_thrust(rocket_id, rcs_link_idx, [thrust, 0, 0], rcs_offset_left, p.LINK_FRAME)
            elif F_pid[0] < 0:
                thrust = min(-F_pid[0], max_rcs)
                apply_thrust(rocket_id, rcs_link_idx, [-thrust, 0, 0], rcs_offset_right, p.LINK_FRAME)
            # Sumbu Y:
            if F_pid[1] > 0:
                thrust = min(F_pid[1], max_rcs)
                apply_thrust(rocket_id, rcs_link_idx, [0, thrust, 0], rcs_offset_back, p.LINK_FRAME)
            elif F_pid[1] < 0:
                thrust = min(-F_pid[1], max_rcs)
                apply_thrust(rocket_id, rcs_link_idx, [0, -thrust, 0], rcs_offset_front, p.LINK_FRAME)

        elif control_mode == 3:
            # ----- Mode 3: Kombinasi Main Engine (dapat diputar) dan RCS -----
            alpha = 0.5  # fraksi yang dialokasikan ke main engine untuk koreksi horizontal
            F_main_x = alpha * F_pid[0]
            F_main_y = alpha * F_pid[1]
            F_main_z = F_pid[2]  # main engine fokus pada vertikal
            F_total = math.sqrt(F_main_x**2 + F_main_y**2 + F_main_z**2)
            if F_total > max_main:
                scale = max_main / F_total
                F_main_x *= scale
                F_main_y *= scale
                F_main_z *= scale
                F_total = max_main
            if F_total < 1e-3:
                d = [0, 0, 1]
            else:
                d = [F_main_x/F_total, F_main_y/F_total, F_main_z/F_total]
            engine_angle = math.asin(d[0])
            plate_angle = -math.atan2(d[1], d[2])
            p.setJointMotorControl2(rocket_id, plate_joint_idx,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=plate_angle, force=10)
            p.setJointMotorControl2(rocket_id, engine_joint_idx,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=engine_angle, force=10)
            apply_thrust(rocket_id, main_engine_link_idx, [0, 0, F_total], [0, 0, 0], p.LINK_FRAME)
            # Sisa koreksi horizontal dialokasikan ke RCS.
            F_rcs_x = (1 - alpha) * F_pid[0]
            F_rcs_y = (1 - alpha) * F_pid[1]
            if F_rcs_x > 0:
                thrust = min(F_rcs_x, max_rcs)
                apply_thrust(rocket_id, rcs_link_idx, [thrust, 0, 0], rcs_offset_left, p.LINK_FRAME)
            elif F_rcs_x < 0:
                thrust = min(-F_rcs_x, max_rcs)
                apply_thrust(rocket_id, rcs_link_idx, [-thrust, 0, 0], rcs_offset_right, p.LINK_FRAME)
            if F_rcs_y > 0:
                thrust = min(F_rcs_y, max_rcs)
                apply_thrust(rocket_id, rcs_link_idx, [0, thrust, 0], rcs_offset_back, p.LINK_FRAME)
            elif F_rcs_y < 0:
                thrust = min(-F_rcs_y, max_rcs)
                apply_thrust(rocket_id, rcs_link_idx, [0, -thrust, 0], rcs_offset_front, p.LINK_FRAME)

        # Update error PID sebelumnya
        e_prev = e[:]
        
        # Update kamera mengikuti posisi roket
        p.resetDebugVisualizerCamera(cameraDistance=2, cameraYaw=45, cameraPitch=-45, cameraTargetPosition=pos)
        p.stepSimulation()
        time.sleep(dt)

if __name__ == "__main__":
    main()


error: Not connected to physics server.

In [1]:
import pybullet as p
import pybullet_data
import time
import math

def main():
    # ----- Setup PyBullet -----
    p.connect(p.GUI)
    #p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.81)
    p.setRealTimeSimulation(0)
    
    # Load plane dan URDF roket
    p.loadURDF("plane.urdf")
    rocket_urdf_path = "LunarLander3DEnv/envs/urdf/rocket.urdf"
    # Posisi awal roket, misal di (0,0,1)
    rocket_start_pos = [0, 0, 1]
    rocket_start_orn = p.getQuaternionFromEuler([0, -math.pi/2, -math.pi/4])
    rocket_id = p.loadURDF(
        rocket_urdf_path,
        rocket_start_pos,
        rocket_start_orn,
        useFixedBase=False
    )
    
    # ----- Identifikasi Joint dan Link -----
    plate_joint_idx = 0
    engine_joint_idx = 1
    main_engine_link_idx = 2
    


    
    # ----- Buat Slider untuk kontrol sederhana -----
    # Slider untuk mengatur rotasi plate (rotasi di sekitar sumbu X)
    slider_plate = p.addUserDebugParameter("Plate Angle (rad)", -math.pi/2, math.pi/2, 0)
    # Slider untuk mengatur rotasi engine (rotasi di sekitar sumbu Y)
    slider_engine = p.addUserDebugParameter("Engine Angle (rad)", -math.pi/2, math.pi/2, 0)
    
    # Gaya thrust konstan, misalnya 150 N
    main_engine_thrust = 0.0

    # Fungsi untuk menerapkan thrust pada main engine
    def apply_thrust(body_id, link_id, force):
        # Gaya diterapkan sepanjang sumbu Z lokal link engine
        p.applyExternalForce(
            objectUniqueId=body_id,
            linkIndex=link_id,
            forceObj=[0, 0, force],
            posObj=[0, 0, 0],
            flags=p.LINK_FRAME
        )

    # ----- Loop Simulasi -----
    dt = 1/240.0
    while True:
        # Baca nilai slider untuk sudut joint
        plate_angle = p.readUserDebugParameter(slider_plate)
        engine_angle = p.readUserDebugParameter(slider_engine)
        #print("Plate angle target:", plate_angle, "Engine angle target:", engine_angle)
        if plate_joint_idx is not None:
            
            p.setJointMotorControl2(
                rocket_id,
                plate_joint_idx,
                controlMode=p.POSITION_CONTROL,
                targetPosition=plate_angle,
                force=100  # force dinaikkan
            )
        if engine_joint_idx is not None:
            
            p.setJointMotorControl2(
                rocket_id,
                engine_joint_idx,
                controlMode=p.POSITION_CONTROL,
                targetPosition=engine_angle,
                force=100  # force dinaikkan
            )
        
        # Terapkan thrust konstan ke main engine (mengikuti orientasi engine)
        if main_engine_link_idx is not None:
            apply_thrust(rocket_id, main_engine_link_idx, main_engine_thrust)
        
        # Update simulasi dan kamera (opsional)
        pos, _ = p.getBasePositionAndOrientation(rocket_id)
        p.resetDebugVisualizerCamera(cameraDistance=1, cameraYaw=45, cameraPitch=-0, cameraTargetPosition=pos)
        p.stepSimulation()
        time.sleep(dt)

if __name__ == "__main__":
    main()


error: Not connected to physics server.

In [1]:
import pybullet as p
import pybullet_data
import time
import math

def main():
    p.connect(p.GUI)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.resetSimulation()
    p.setGravity(0, 0, -9.81)
    
    # Muat plane
    p.loadURDF("plane.urdf")
    
    # Pastikan file scara.urdf berada di folder yang sama dengan script atau update path-nya
    scara_urdf = "scara.urdf"
    start_pos = [0, 0, 0]
    start_orn = p.getQuaternionFromEuler([0, 0, 0])
    robot_id = p.loadURDF(scara_urdf, start_pos, start_orn, useFixedBase=True)
    
    # Buat slider untuk kontrol joint
    joint1_slider = p.addUserDebugParameter("Joint1 (rad)", -math.pi, math.pi, 0)
    joint2_slider = p.addUserDebugParameter("Joint2 (rad)", -math.pi, math.pi, 0)
    
    dt = 1/240.0
    while True:
        # Baca nilai target dari slider
        target_joint1 = p.readUserDebugParameter(joint1_slider)
        target_joint2 = p.readUserDebugParameter(joint2_slider)
        
        # Gunakan position control untuk masing-masing joint
        # Dalam URDF, joint urutan pertama (index 0) adalah joint1 dan index 1 adalah joint2.
        p.setJointMotorControl2(robot_id, 0, p.POSITION_CONTROL, targetPosition=target_joint1, force=50)
        p.setJointMotorControl2(robot_id, 1, p.POSITION_CONTROL, targetPosition=target_joint2, force=50)
        
        p.stepSimulation()
        time.sleep(dt)

if __name__ == "__main__":
    main()


error: Not connected to physics server.

In [1]:
import pybullet as p
import pybullet_data
import time
import math

def main():
    # ----- Setup PyBullet -----
    p.connect(p.GUI)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.resetSimulation()
    p.setGravity(0, 0, -9.81)
    
    # Muat plane (lantai)
    p.loadURDF("plane.urdf")
    
    # Pastikan file URDF engine_xy.urdf berada di folder yang sesuai,
    # atau gunakan path absolut jika diperlukan.
    urdf_path = "engine_xy.urdf"
    start_pos = [0, 0, 0.1]
    start_orn = p.getQuaternionFromEuler([0, 0, 0])
    robot_id = p.loadURDF(urdf_path, start_pos, start_orn, useFixedBase=True)
    
    # Buat slider untuk mengontrol joint
    slider_plate = p.addUserDebugParameter("Plate Joint (rad)", -math.pi/2, math.pi/2, 0)
    slider_ring = p.addUserDebugParameter("Ring Joint (rad)", -math.pi/2, math.pi/2, 0)
    
    dt = 1/240.0
    while True:
        # Baca nilai target dari slider
        target_plate = p.readUserDebugParameter(slider_plate)
        target_ring = p.readUserDebugParameter(slider_ring)
        
        # Dalam URDF, joint index 0 adalah plate_joint dan index 1 adalah ring_joint
        p.setJointMotorControl2(robot_id, 0, p.POSITION_CONTROL, targetPosition=target_plate, force=100)
        p.setJointMotorControl2(robot_id, 1, p.POSITION_CONTROL, targetPosition=target_ring, force=100)
        
        p.stepSimulation()
        time.sleep(dt)

if __name__ == "__main__":
    main()


error: Cannot load URDF file.

In [1]:
import pybullet as p
import pybullet_data
import time
import math

def main():
    # ----- Setup PyBullet -----
    p.connect(p.GUI)
    #p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.resetSimulation()
    p.setGravity(0, 0, -9.81)
    p.setRealTimeSimulation(0)
    
    # Load plane dan URDF roket
    p.loadURDF("plane.urdf")
    rocket_urdf_path = "LunarLander3DEnv/envs/urdf/rocket.urdf"
    rocket_start_pos = [0, 0, 0]  # atur posisi awal sesuai kebutuhan
    rocket_start_orn = p.getQuaternionFromEuler([0, 0, 0])
    rocket_id = p.loadURDF(
        rocket_urdf_path,
        rocket_start_pos,
        rocket_start_orn,
        useFixedBase=False
    )
    
    # ----- Identifikasi Joint dan Link -----
    plate_joint_idx = None
    engine_joint_idx = None
    num_joints = p.getNumJoints(rocket_id)
    for i in range(num_joints):
        info = p.getJointInfo(rocket_id, i)
        joint_name = info[1].decode("utf-8")
        if joint_name == "plate_joint":
            plate_joint_idx = i
        elif joint_name == "engine_joint":
            engine_joint_idx = i

    main_engine_link_idx = None
    rcs_link_idx = None
    for i in range(num_joints):
        info = p.getJointInfo(rocket_id, i)
        link_name = info[12].decode("utf-8")
        if link_name == "engine":
            main_engine_link_idx = i
        elif link_name == "rcs":
            rcs_link_idx = i

    # ----- Buat Slider untuk Kontrol Manual -----
    # Slider untuk mengatur thrust utama (vertikal) dalam Newton
    slider_main = p.addUserDebugParameter("Main Engine Thrust (N)", 0, 200, 100)
    # Slider untuk mengatur thrust horizontal sumbu X (N) - nilai positif dan negatif
    slider_rcs_x = p.addUserDebugParameter("RCS Thrust X (N)", -10, 10, 0)
    # Slider untuk mengatur thrust horizontal sumbu Y (N) - nilai positif dan negatif
    slider_rcs_y = p.addUserDebugParameter("RCS Thrust Y (N)", -10, 10, 0)
    
    # Untuk mode 2: pastikan main engine fixed (tidak diputar)
    if plate_joint_idx is not None:
        p.setJointMotorControl2(
            rocket_id,
            plate_joint_idx,
            controlMode=p.POSITION_CONTROL,
            targetPosition=0,
            force=100
        )
    if engine_joint_idx is not None:
        p.setJointMotorControl2(
            rocket_id,
            engine_joint_idx,
            controlMode=p.POSITION_CONTROL,
            targetPosition=0,
            force=100
        )
    
    # ----- Definisikan Offset untuk Thruster RCS -----
    # Asumsi rcs berbentuk plat bundar dengan radius sekitar 0.05 m
    rcs_offset_left  = [-0.05, 0, 0]   # jika thrust X positif, gunakan thruster di sisi kiri (menghasilkan gaya +X)
    rcs_offset_right = [ 0.05, 0, 0]   # jika thrust X negatif, gunakan thruster di sisi kanan
    rcs_offset_front = [0,  0.05, 0]    # jika thrust Y negatif, gunakan thruster di sisi depan
    rcs_offset_back  = [0, -0.05, 0]    # jika thrust Y positif, gunakan thruster di sisi belakang
    
    # ----- Fungsi untuk Menerapkan Thrust -----
    def apply_thrust(body_id, link_id, force_dir, pos_offset=[0, 0, 0], frame=p.LINK_FRAME):
        p.applyExternalForce(
            objectUniqueId=body_id,
            linkIndex=link_id,
            forceObj=force_dir,
            posObj=pos_offset,
            flags=frame
        )
    
    dt = 1/240.0
    while True:
        # Baca nilai dari slider
        main_thrust = p.readUserDebugParameter(slider_main)
        rcs_x = p.readUserDebugParameter(slider_rcs_x)
        rcs_y = p.readUserDebugParameter(slider_rcs_y)
        
        # Terapkan thrust utama pada main engine (arah lokal +Z)
        if main_engine_link_idx is not None:
            apply_thrust(rocket_id, main_engine_link_idx, [0, 0, main_thrust], [0, 0, 0], p.LINK_FRAME)
        
        # Terapkan thrust RCS untuk koreksi sumbu X
        if rcs_link_idx is not None:
            if rcs_x > 0:
                # Jika rcs_x positif, gunakan thruster di sisi kiri untuk menghasilkan gaya +X
                apply_thrust(rocket_id, rcs_link_idx, [rcs_x, 0, 0], rcs_offset_left, p.LINK_FRAME)
            elif rcs_x < 0:
                apply_thrust(rocket_id, rcs_link_idx, [rcs_x, 0, 0], rcs_offset_right, p.LINK_FRAME)
            
            # Terapkan thrust RCS untuk koreksi sumbu Y
            if rcs_y > 0:
                apply_thrust(rocket_id, rcs_link_idx, [0, rcs_y, 0], rcs_offset_back, p.LINK_FRAME)
            elif rcs_y < 0:
                apply_thrust(rocket_id, rcs_link_idx, [0, rcs_y, 0], rcs_offset_front, p.LINK_FRAME)
        
        pos, _ = p.getBasePositionAndOrientation(rocket_id)
        p.resetDebugVisualizerCamera(cameraDistance=1, cameraYaw=45, cameraPitch=-0, cameraTargetPosition=pos)
        p.stepSimulation()
        time.sleep(dt)

if __name__ == "__main__":
    main()


error: Not connected to physics server.

In [ ]:
import pybullet as p
import pybullet_data
import time
import math

# Hubungkan ke PyBullet (GUI mode)
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Atur parameter simulasi
p.setGravity(0, 0, -9.81)
p.setTimeStep(1.0/240.0)

# Muat plane dan robot URDF
planeId = p.loadURDF("plane.urdf")
# Pastikan file "humanoid_50cm.urdf" berada di direktori kerja yang sama
robotId = p.loadURDF("humanoid.urdf", basePosition=[0, 0, 0.5])

# Tampilkan daftar sendi yang terdeteksi
num_joints = p.getNumJoints(robotId)
print("Number of joints:", num_joints)
for i in range(num_joints):
    joint_info = p.getJointInfo(robotId, i)
    joint_name = joint_info[1].decode('utf-8')
    joint_type = joint_info[2]
    print("Joint index:", i, "Name:", joint_name, "Type:", joint_type)

# Simulasi loop: Terapkan perintah kontrol sinusoidal pada semua sendi revolute
t0 = time.time()
while time.time() - t0 < 100:  # Simulasi selama 10 detik
    current_time = time.time()
    for i in range(num_joints):
        joint_info = p.getJointInfo(robotId, i)
        joint_type = joint_info[2]
        # Hanya terapkan kontrol pada sendi revolute (bukan fixed)
        if joint_type == p.JOINT_REVOLUTE:
            # Amplitudo 0.5 radian, frekuensi 2Hz sebagai contoh
            target_angle = 0.5 * math.sin(2.0 * current_time)
            p.setJointMotorControl2(bodyIndex=robotId,
                                    jointIndex=i,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=target_angle,
                                    force=10)
    p.stepSimulation()
    time.sleep(1.0/240.0)

p.disconnect()


Number of joints: 5
Joint index: 0 Name: body_to_engine Type: 4
Joint index: 1 Name: body_to_fin1 Type: 4
Joint index: 2 Name: body_to_fin2 Type: 4
Joint index: 3 Name: body_to_fin3 Type: 4
Joint index: 4 Name: body_to_fin4 Type: 4


In [1]:
import pybullet as p
import pybullet_data
import time

# Inisialisasi PyBullet
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetSimulation()
p.setGravity(0, 0, -9.81)

# Muat plane dan model starship
plane_id = p.loadURDF("plane.urdf")
starship_id = p.loadURDF("starship.urdf", basePosition=[0, 0, 0.5])

# Mapping nama joint dan link untuk kemudahan referensi
num_joints = p.getNumJoints(starship_id)
joint_name_to_index = {}
link_name_to_index = {}

for i in range(num_joints):
    joint_info = p.getJointInfo(starship_id, i)
    joint_name = joint_info[1].decode('utf-8')
    link_name = joint_info[12].decode('utf-8')
    joint_name_to_index[joint_name] = i
    link_name_to_index[link_name] = i

# --- Slider untuk kontrol ---
# Gimbal thruster utama
gimbal_y_slider = p.addUserDebugParameter("Gimbal Y (rad)", -0.5, 0.5, 0.0)
gimbal_x_slider = p.addUserDebugParameter("Gimbal X (rad)", -0.5, 0.5, 0.0)
# Gaya mesin utama
main_engine_slider = p.addUserDebugParameter("Main Engine Thrust (N)", 0, 1000, 0)
# RCS untuk pitch dan roll
rcs_front_slider = p.addUserDebugParameter("RCS Front (N)", 0, 100, 0)
rcs_back_slider  = p.addUserDebugParameter("RCS Back (N)", 0, 100, 0)
rcs_left_slider  = p.addUserDebugParameter("RCS Left (N)", 0, 100, 0)
rcs_right_slider = p.addUserDebugParameter("RCS Right (N)", 0, 100, 0)
# RCS untuk yaw
rcs_yaw_left_slider  = p.addUserDebugParameter("RCS Yaw Left (N)", 0, 100, 0)
rcs_yaw_right_slider = p.addUserDebugParameter("RCS Yaw Right (N)", 0, 100, 0)

# Fungsi untuk mengaplikasikan gaya pada link tertentu
def apply_force_to_link(link_name, force_vector):
    link_index = link_name_to_index.get(link_name, -1)
    if link_index != -1:
        p.applyExternalForce(objectUniqueId=starship_id,
                             linkIndex=link_index,
                             forceObj=force_vector,
                             posObj=[0, 0, 0],
                             flags=p.LINK_FRAME)

# Loop simulasi utama
while p.isConnected():
    # --- Update kontrol gimbal ---
    gimbal_y_angle = p.readUserDebugParameter(gimbal_y_slider)
    gimbal_x_angle = p.readUserDebugParameter(gimbal_x_slider)
    
    if "gimbal_y" in joint_name_to_index:
        p.setJointMotorControl2(bodyUniqueId=starship_id,
                                jointIndex=joint_name_to_index["gimbal_y"],
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=gimbal_y_angle,
                                force=10)
    if "gimbal_x" in joint_name_to_index:
        p.setJointMotorControl2(bodyUniqueId=starship_id,
                                jointIndex=joint_name_to_index["gimbal_x"],
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=gimbal_x_angle,
                                force=10)
    
    # --- Update gaya mesin utama ---
    main_thrust = p.readUserDebugParameter(main_engine_slider)
    # Terapkan gaya pada link 'thruster' sepanjang sumbu Z lokal
    p.applyExternalForce(objectUniqueId=starship_id,
                         linkIndex=link_name_to_index.get("thruster", -1),
                         forceObj=[0, 0, main_thrust],
                         posObj=[0, 0, 0],
                         flags=p.LINK_FRAME)
    
    # --- Update gaya RCS untuk pitch & roll ---
    rcs_front_force = p.readUserDebugParameter(rcs_front_slider)
    rcs_back_force  = p.readUserDebugParameter(rcs_back_slider)
    rcs_left_force  = p.readUserDebugParameter(rcs_left_slider)
    rcs_right_force = p.readUserDebugParameter(rcs_right_slider)
    
    # RCS Front: misal menghasilkan gaya ke arah -Y
    apply_force_to_link("rcs_front", [0, -rcs_front_force, 0])
    # RCS Back: menghasilkan gaya ke arah +Y
    apply_force_to_link("rcs_back", [0, rcs_back_force, 0])
    # RCS Left: menghasilkan gaya ke arah +X
    apply_force_to_link("rcs_left", [rcs_left_force, 0, 0])
    # RCS Right: menghasilkan gaya ke arah -X
    apply_force_to_link("rcs_right", [-rcs_right_force, 0, 0])
    
    # --- Update gaya RCS untuk yaw ---
    rcs_yaw_left_force  = p.readUserDebugParameter(rcs_yaw_left_slider)
    rcs_yaw_right_force = p.readUserDebugParameter(rcs_yaw_right_slider)
    # Untuk menghasilkan torsi yaw, kita aplikasikan gaya yang bekerja sejajar dengan sumbu Y.
    # Misal: rcs_yaw_left mendorong ke arah +Y dan rcs_yaw_right mendorong ke arah -Y.
    apply_force_to_link("rcs_yaw_left", [0, rcs_yaw_left_force, 0])
    apply_force_to_link("rcs_yaw_right", [0, -rcs_yaw_right_force, 0])
    
    p.stepSimulation()
    time.sleep(1./240.)


In [3]:
import pybullet as p
import pybullet_data
import time
import math
import matplotlib.pyplot as plt

# Kelas PID untuk kontrol
class PID:
    def __init__(self, Kp, Ki, Kd, setpoint=0.0):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.setpoint = setpoint
        self.integral = 0.0
        self.prev_error = 0.0

    def update(self, measurement, dt):
        error = self.setpoint - measurement
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        self.prev_error = error
        return self.Kp * error + self.Ki * self.integral + self.Kd * derivative

# Setpoint dan nilai yang diinginkan
desired_vz    = 10.0                # Kecepatan vertikal naik (m/s)
desired_pitch = math.radians(10)    # Pitch 10 derajat
desired_roll  = math.radians(10)    # Roll 10 derajat
desired_yaw   = 0.0                 # Yaw stabil (0 derajat)

# Parameter PID (nilai awal, perlu tuning lebih lanjut)
# PID untuk kecepatan vertikal (main engine)
Kp_thrust = 100.0
Ki_thrust = 0.0
Kd_thrust = 20.0

# PID untuk pitch (mengontrol gimbal_y)
Kp_pitch = 1.0
Ki_pitch = 0.0
Kd_pitch = 0.2

# PID untuk roll (mengontrol gimbal_x)
Kp_roll = 1.0
Ki_roll = 0.0
Kd_roll = 0.2

# PID untuk yaw (mengontrol RCS yaw)
Kp_yaw = 1.0
Ki_yaw = 0.0
Kd_yaw = 0.2

# Inisialisasi controller PID
pid_thrust = PID(Kp_thrust, Ki_thrust, Kd_thrust, setpoint=desired_vz)
pid_pitch  = PID(Kp_pitch, Ki_pitch, Kd_pitch, setpoint=desired_pitch)
pid_roll   = PID(Kp_roll, Ki_roll, Kd_roll, setpoint=desired_roll)
pid_yaw    = PID(Kp_yaw, Ki_yaw, Kd_yaw, setpoint=desired_yaw)

# Inisialisasi simulasi PyBullet
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetSimulation()
p.setGravity(0, 0, -9.81)

# Muat plane dan model roket
plane_id = p.loadURDF("plane.urdf")
starship_id = p.loadURDF("starship.urdf", basePosition=[0, 0, 0.5])

# Mapping nama joint dan link untuk kemudahan referensi
num_joints = p.getNumJoints(starship_id)
joint_name_to_index = {}
link_name_to_index = {}

for i in range(num_joints):
    joint_info = p.getJointInfo(starship_id, i)
    joint_name = joint_info[1].decode('utf-8')
    link_name = joint_info[12].decode('utf-8')
    joint_name_to_index[joint_name] = i
    link_name_to_index[link_name] = i

# Fungsi untuk mengaplikasikan gaya pada link tertentu dengan frame lokal
def apply_force_to_link(link_name, force_vector):
    link_index = link_name_to_index.get(link_name, -1)
    if link_index != -1:
        p.applyExternalForce(objectUniqueId=starship_id,
                             linkIndex=link_index,
                             forceObj=force_vector,
                             posObj=[0, 0, 0],
                             flags=p.LINK_FRAME)

# Data untuk plotting (opsional)
time_data     = []
vz_data       = []
pitch_data  = []
roll_data   = []
yaw_data    = []
thrust_cmd_data = []
pitch_cmd_data  = []
roll_cmd_data   = []
yaw_cmd_data    = []

sim_duration = 20.0  # durasi simulasi dalam detik
dt = 1.0/240.0
num_steps = int(sim_duration/dt)

# Loop simulasi utama
for step in range(num_steps):
    # Dapatkan posisi, orientasi, dan kecepatan
    pos, orn = p.getBasePositionAndOrientation(starship_id)
    vel, ang_vel = p.getBaseVelocity(starship_id)
    euler = p.getEulerFromQuaternion(orn)  # (roll, pitch, yaw)

    # Ambil kecepatan vertikal saat ini (z)
    current_vz = vel[2]
    # Update PID untuk thrust
    thrust_cmd = pid_thrust.update(current_vz, dt)
    thrust_cmd = max(0, min(thrust_cmd, 1000))  # Batasan thrust antara 0 dan 1000 N

    # Update PID untuk orientasi
    pitch_cmd = pid_pitch.update(euler[1], dt)
    roll_cmd  = pid_roll.update(euler[0], dt)
    yaw_cmd   = pid_yaw.update(euler[2], dt)

    # Terapkan gaya mesin utama pada link 'thruster' (mengarah ke atas dalam frame lokal)
    p.applyExternalForce(objectUniqueId=starship_id,
                         linkIndex=link_name_to_index.get("thruster", -1),
                         forceObj=[0, 0, thrust_cmd],
                         posObj=[0, 0, 0],
                         flags=p.LINK_FRAME)

    # Kontrol gimbal untuk pitch dan roll
    if "gimbal_y" in joint_name_to_index:
        # gimbal_y digunakan untuk mengatur pitch
        p.setJointMotorControl2(bodyUniqueId=starship_id,
                                jointIndex=joint_name_to_index["gimbal_y"],
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=pitch_cmd,
                                force=10)
    if "gimbal_x" in joint_name_to_index:
        # gimbal_x digunakan untuk mengatur roll
        p.setJointMotorControl2(bodyUniqueId=starship_id,
                                jointIndex=joint_name_to_index["gimbal_x"],
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=roll_cmd,
                                force=10)

    # Terapkan RCS untuk kontrol yaw
    # Misalnya, jika yaw_cmd positif, rcs_yaw_left diberi gaya ke +Y dan rcs_yaw_right ke -Y
    apply_force_to_link("rcs_yaw_left", [0, yaw_cmd, 0])
    apply_force_to_link("rcs_yaw_right", [0, -yaw_cmd, 0])

    # Simpan data untuk analisis/tuning
    current_time = step * dt
    time_data.append(current_time)
    vz_data.append(current_vz)
    pitch_data.append(euler[1])
    roll_data.append(euler[0])
    yaw_data.append(euler[2])
    thrust_cmd_data.append(thrust_cmd)
    pitch_cmd_data.append(pitch_cmd)
    roll_cmd_data.append(roll_cmd)
    yaw_cmd_data.append(yaw_cmd)

    p.stepSimulation()
    time.sleep(dt)

# Plot hasil kontrol untuk analisis
plt.figure(figsize=(12, 10))

plt.subplot(3, 1, 1)
plt.plot(time_data, vz_data, label="Kecepatan Vertikal (m/s)")
plt.plot(time_data, thrust_cmd_data, label="Perintah Thrust (N)")
plt.xlabel("Waktu (s)")
plt.ylabel("Nilai")
plt.title("Kontrol Kecepatan Vertikal")
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(time_data, [math.degrees(a) for a in pitch_data], label="Pitch (deg)")
plt.plot(time_data, [math.degrees(a) for a in roll_data], label="Roll (deg)")
plt.xlabel("Waktu (s)")
plt.ylabel("Sudut (derajat)")
plt.title("Kontrol Pitch & Roll")
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(time_data, [math.degrees(a) for a in yaw_data], label="Yaw (deg)")
plt.plot(time_data, yaw_cmd_data, label="Perintah Yaw")
plt.xlabel("Waktu (s)")
plt.ylabel("Sudut (deg)")
plt.title("Kontrol Yaw")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

p.disconnect()


error: Not connected to physics server.